# --------TAREA 1 CONFIGURACION DE ENTORNO Y PROCESO ETL INICIOS ------


**MAESTRIA EN CIBERSEGURIDAD**

**INTELIGENCIA DE NEGOCIOS**

**INTEGRANTES: JULIO AREVALO, EMILIO CORDOVA, NILSON ROMERO**

**2025**

## RESUMEN 
En este Notebook podrá encontrar el proceso completo de configuración de un entorno de análisis de datos y la ejecución de pipeline ETL. Se extraen datos de diferentes fuentes como PostgreSQL, CSV, JSON.
Se procede a cargar data Warehouse en PostgreSQL y se realiza una explotación y segmentación inicial de los datos utilizando librerías como Pandas en Python. 

# ----------IMPORTANCION DE LIBRERIAS CONEXION POSTGRESQL----------

Se inicia la actividad estableciendo la conexión a la base de datos de destino, esta DB funcionara como un **Data Warehouse**.
Utilizaremos como motor *DB* **PostgreSQL** esta base estará alojada en un contenedor **Docker**. Esto garantiza un entorno de desarrollo consistente, aislable y reproducible.
Se definen los parámetros de conexión y se creará el *motor* *(engine)* utilizando las librería *SQLalchemy*. El motor actuará como puente entre nuestra herramienta de análisis en Python y la base de datos.
En este proceso se ha implementado un bloque **try-except** para gestionar de forma precisa cualquier posible error durante el intento de conexión.

In [ ]:
# ----- IMPORTACION DE LIBRERIAS -----
# Se importan las librerías necesarias para la manipulación de datos y para la conexión con la base de datos 
# qué contendrá la data.

import pandas as pd
from sqlalchemy import create_engine # crea la conexion a la base de datos
import os # proporciona funciones para interactuar con el sistema operativo (manejar rutas de archivos)
import json # para trabajar con datos en formato JSON
from dotenv import load_dotenv # para cargar variables de entorno desde un archivo .env
# ----- CONEXION A LA BASE DE DATOS POSTGRESQL EN DOCKER -----
# --- 2. CARGAR Y LEER VARIABLES DE ENTORNO ---
# La función load_dotenv() buscará y cargará las variables del archivo .env
load_dotenv()
print("Variables de entorno cargadas.")

# Leemos cada variable del entorno usando os.getenv()
DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")

#-----Creación del motor de conexión SQLalchemy-----
#se utiliza un bloque try-except para el manejo de errores.
#Si la conexión falla el programa informará del error en lugar de tenerse abruptamente.
# --- 3. CONEXIÓN A LA BASE DE DATOS ---
print("Iniciando conexión a la base de datos...")
try:
    # La cadena de conexión ahora se construye con las variables cargadas, sin secretos en el código.
    engine = create_engine(f'postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}')

    # Verificamos la conexión para asegurar que todo funcionó
    with engine.connect() as connection:
        print("¡Conexión a PostgreSQL exitosa!")

except Exception as e:
    print(f"---> ERROR al conectar: {e}")   



En la salida nos indica que el proceso de conexion ha sido exitoso 


### CARGA MASIVA DE CSV A LA TABLA DE POSTGRESQL PROCESO ETL

En esta sección del código se podrá encontrar el núcleo del proceso **ETL** para inyectar la base de datos **postgreSQL**.
La consigna es consolidar múltiples archivos **CSV** relacionados en un único **data warehouse** y posteriormente se realizará el análisis de la información.

Se describe los pasos:

**Selección de la fuente EXTRACT:** Procedemos a definir una lista explícita de los archivos **CSV** que contienen las entidades del negocio las principales *(clientes, órdenes, productos)*

**Bucle de procesamiento:** se itegra sobre cada archivo de la lista para procesarlo de manera automatizada.

**Transformación ligera:** dentro del bucle se realiza una transformación básica para generar nombres de tabla limpios el lógicos a partir de los nombres de los archivos. ej. `olist_customers_dataset.csv` se convertirá en la tabla `customers`.

**Carga:** se utiliza la función **to_sql** de *Pandas* para cargar los datos de cada archivo en su tabla correspondiente dentro del progress SQL. Se ha configurado para reemplazar la tabla si ya existe.



In [ ]:
# Definimos la ruta donde se encuentran los archivos CSV descargados

data_path = r'C:\Users\j_dar\downloads\archive'

# SELECCIONAMOS LOS ARCHIVOS CSV
# Se crea una lista con los nombres de los archivos CSV que se van a cargar en la base de datos.
files_to_load_in_db = [
    'olist_customers_dataset.csv',
    'olist_orders_dataset.csv',
    'olist_order_items_dataset.csv',
    'olist_products_dataset.csv',
    'olist_sellers_dataset.csv'
]

#----- Blucle para cargar los archivos CSV en la base de datos -----
# INICIAMOS EL PROCESO DE CARGA MASIVA DE LOS ARCHIVOS CSV A LA BASE DE DATOS

print("\nIniciando carga de datos a PostgreSQL...")
for file in files_to_load_in_db:
    # El nombre de la tabla será el del archivo sin la extensión y prefijos
    table_name = file.replace('olist_', '').replace('_dataset.csv', '')
    df = pd.read_csv(os.path.join(data_path, file))
    df.to_sql(table_name, engine, if_exists='replace', index=False)
    print(f"Tabla '{table_name}' cargada con {len(df)} filas.")
print("Carga a PostgreSQL completada.")

### GENERACION DE LOS DATAFRAME PRINCIPALES
En este proceso procederemos a realizar la creacion de los dataframes con la libreria pandas.

Se procede con la **extracción de datos desde tres fuentes distintas**, generando un **DataFrame** para cada una de ellas.
Este proceso de simulación va a reflejar un escenario empresarial *real* dónde la información reside en un sistema.

**DataFrame creados:**
**`df_orders` (Desde PostgreSQL ):** Se consultará la tabla de nuestro **DataWarehouse** esta presenta la fuente de datos principales ya consolidada y estructura.
**`df_order_reviews` (Desde CSV ):** Se leerá un archivo **CSV** externo, esto simula la ingesta de datos provenientes de un reporte externo o un sistema *Legacy*.
**`df_json` (Desde JSON):** Se simula una fuente `JSON` el proceso consistirá en convertir primero un archivo `csv` de referencia a un archivo `JSON` Y luego leerlo.


In [ ]:
#----- Creacion del dataframe 1 desde PostGreSQL-----

print("1. Creando dataframe desde PostgreSQL tabla 'orders'")
query_orders = "SELECT * FROM orders;"
df_orders = pd.read_sql(query_orders, engine)

# ----- Creacion del dataframe 2 desde CSV -----

print("2. CREACION DE DATAFRAME DESDE CSV ('ORDER_REVIWS') ")
df_order_reviews = pd.read_csv(os.path.join(data_path, 'olist_order_reviews_dataset.csv'))

#----- creacion del dataframe 3 desde JSON -----

print("3. CREACION DE DATAFRAME DESDE JSON ('product_category_translation') ")

# convertir el csv a un archivo JSON PARA SIMULAR LA FUENTE
df_temp_json = pd.read_csv(os.path.join(data_path, 'product_category_name_translation.csv'))
json_path = os.path.join(data_path, 'categories.json')
df_temp_json.to_json(json_path, orient='records', lines=True)

# leer el archivo JSON
df_json = pd.read_json(json_path, lines=True)

print("LOS DATAFRAME PRINCIPALES HAN SIDO CREADOS EXITOSAMENTE")


# EXPLORACION Y FILTROS DEL DATAFRAME  DE POSTGRESQL DF_POSTGRES

Una vez que se ha procedido a cargar los **DataFrame** el siguiente paso es realizar el ** análisis exploratorio de datos EDA** este paso es fundamental para comprender la naturaleza de nuestra información antes de realizar otro tipo de análisis que sean más complejos.

Se inicia con él **DataFrame** `df_orders`, Que contiene la información de las órdenes de compra en nuestra base de datos **PostgreSQL**

**Se ha dividido el análisis en dos partes:**

**Exploración inicial:** Se utilizarán los métodos `.info()` y `.head()` de `Pandas` para responder 

**Filtrado y Segmentación:** Se aplicarán tres filtros específicos para segmentar los datos y responder a preguntas de negocio iniciales.

In [ ]:
# Exploracion de los dataframes creados
# Exploramos los dataframes creados para entender su estructura y contenido.

print("exploracion del postgreSQL df_postgres (orders)")
print(df_orders.info())
print("\nPrimeras cinco filas:")
display(df_orders.head())


In [ ]:
# FILTRO PEDIDOS ENTREGADOS
entregados = df_orders[df_orders['order_status'] == 'delivered']
print(f"HAY {len(entregados)} PEDIDOS ENTREGADOS")

# PEDIDOS REALIZADOS EN EL 2017
pedidos_2017 = df_orders[pd.to_datetime(df_orders['order_purchase_timestamp']).dt.year == 2017]
print(f"hay {len(pedidos_2017)} pedidos realizados en el 2017")
pedidos_2017


In [ ]:
# ----- PEDIDOS CANCELADOS O NO DISPONIBLES -----
cancelados = df_orders[df_orders['order_status'].isin(['canceled', 'unavailable'])]
print(f"HAY {len(cancelados)} PEDIDOS CANCELADOS O NO DISPONIBLES")
cancelados

### EXPLORACION Y FILTROS DATAFRAME CSV

Esta vez enfocándonos en el DataFrame `df_order_reviews`, que fue cargado desde nuestra fuente CSV obligatoria.

Estos datos son **crítico para entender la satisfacción del cliente**, ya que contiene las puntuaciones y comentarios que los usuarios han dejado después de una compra.



**El proceso es el sigguiente:**

**Exploración de la Estructura de Datos** 

**Interpretación de Hallazgos Preliminares**

Se analizará la salida de la exploración para identificar puntos clave, como la cantidad de comentarios escritos versus reseñas sin texto, y la necesidad de convertir tipos de datos para análisis futuros.

In [ ]:
print("----- exploracion de df_csv (reviews) -----")
print(df_order_reviews.info())
print("\nPrimeras cinco filas:")
display(df_order_reviews.head())

In [ ]:
## ----- reseñas con una puntuación de cinco estrellas -----
resenias_5_estrellas = df_order_reviews[df_order_reviews['review_score'] == 5]
print(f"HAY {len(resenias_5_estrellas)} RESEÑAS CON 5 ESTRELLAS")
resenias_5_estrellas

In [ ]:
## ----- RESEÑAS CON COMENTARIOS -----
resenias_con_comentarios = df_order_reviews[df_order_reviews['review_comment_message'].notnull() & (df_order_reviews['review_comment_message'].str.strip() != '')]
print(f"HAY {len(resenias_con_comentarios)} RESEÑAS CON COMENTARIOS")
resenias_con_comentarios

In [ ]:
## RESEÑAS CON LA MINIMAPUNTUACIÓN 1 ESTRELLA
resenias_1_estrella = df_order_reviews[df_order_reviews['review_score'] == 1]
print(f"HAY {len(resenias_1_estrella)} RESEÑAS CON 1 ESTRELLA")
resenias_1_estrella

### ANALISIS DE JSON 
Finalmente, se realiza el Análisis Exploratorio de Datos (EDA) para el DataFrame `df_json`. Este conjunto de datos fue cargado desde la **fuente JSON simulada** y contiene la traducción de los nombres de las categorías de productos del portugués al inglés.


In [ ]:
# EXPLORACIÓN DE DF_JSON 
print("----- Exploración de df_json (categories_translation) -----")
print(df_json.info())
print("\nPrimeras cinco filas:")
display(df_json.head())



In [ ]:
# ----- CATEGORIAS RELACIONADAS CON LA CAMA Y EL BAÑO -----
cama_bano = df_json[df_json['product_category_name_english'] == 'bed_bath_table']
print(f"HAY {len(cama_bano)} CATEGORIAS RELACIONADAS CON LA CAMA Y EL BAÑO")
cama_bano

In [ ]:
# CATEGORIAS RELACIONADAS CON LA PALABRA COMPUTADORA EN PORTUGUES
computadoras_pt = df_json[df_json['product_category_name'].str.contains('informatica', na=False)]
print(f"HAY {len(computadoras_pt)} CATEGORIAS RELACIONADAS CON LA PALABRA COMPUTADORA EN PORTUGUES")
computadoras_pt



In [ ]:
# CATEGORIA REALCIONADA CON EL DEPORTE Y OCIO
deporte_ocio = df_json[df_json['product_category_name_english'] == 'sports_leisure']
print(f"HAY {len(deporte_ocio)} CATEGORIAS RELACIONADAS CON EL DEPORTE Y OCIO")
deporte_ocio